In [219]:
import requests
import pandas as pd
from datetime import date
import pycountry
import plotly.express as px

In [220]:
URL = requests.get("https://travel.state.gov/content/travel/en/traveladvisories/traveladvisories.html/")

In [221]:
df = pd.read_html(URL.text)[0]

In [222]:
df

,Advisory,Level,Date Updated
0,Liechtenstein Travel Advisory,Level 1: Exercise Normal Precautions,"April 19, 2022"
1,North Macedonia Travel Advisory,Level 1: Exercise Normal Precautions,"April 19, 2022"
2,Nauru Travel Advisory,Level 1: Exercise Normal Precautions,"April 18, 2022"
3,Palau Travel Advisory,Level 1: Exercise Normal Precautions,"April 18, 2022"
4,Burma (Myanmar) Travel Advisory,Level 4: Do Not Travel,"April 18, 2022"
...,...,...,...
207,Republic of the Congo Travel Advisory,Level 2: Exercise Increased Caution,"April 19, 2022"
208,Guinea-Bissau Travel Advisory,Level 3: Reconsider Travel,"April 18, 2022"
209,Bonaire Travel Advisory,Level 1: Exercise Normal Precautions,"April 18, 2022"
210,Sint Eustatius Travel Advisory,Level 1: Exercise Normal Precautions,"April 18, 2022"


In [223]:
df.sort_values(by=['Level'], inplace=True)

In [224]:
df["Advisory"] = df["Advisory"].str.replace(" Travel Advisory", "")

In [225]:
df.rename(columns={"Advisory": "Country"}, inplace=True)

In [226]:
df["Date Updated"] = pd.to_datetime(df["Date Updated"])
df["Date Updated"] = df["Date Updated"].dt.strftime('%Y-%m-%d')

In [227]:
df

,Country,Level,Date Updated
5,Worldwide Caution,Caution,2019-01-15
0,Liechtenstein,Level 1: Exercise Normal Precautions,2022-04-19
111,Malawi,Level 1: Exercise Normal Precautions,2022-04-18
210,Sint Eustatius,Level 1: Exercise Normal Precautions,2022-04-18
104,Kuwait,Level 1: Exercise Normal Precautions,2022-04-20
...,...,...,...
160,Ukraine,Level 4: Do Not Travel,2022-04-19
164,Venezuela,Level 4: Do Not Travel,2022-04-18
27,Hong Kong,Level 4: Do Not Travel,2022-04-25
146,Sudan,Level 4: Do Not Travel,2022-04-19


In [228]:
today = date.today().strftime("%Y-%m-%d")

In [229]:
df.to_excel("Travel Advisories " + today + ".xlsx", index=False)

In [230]:
df.replace('Burma (Myanmar)', 'Myanmar', inplace=True)
df.replace('Brunei', 'Brunei Darussalam', inplace=True)
df.replace('Macau', 'Macao', inplace=True)
df.replace('Curacao', 'Curaçao', inplace=True)
df.replace('Sint Maarten', 'Sint Maarten (Dutch part)', inplace=True)
df.replace('Iran', 'Iran, Islamic Republic of', inplace=True)
df.replace('Israel, The West Bank and Gaza', 'Israel', inplace=True)
df.replace('North Korea (Democratic People\'s Republic of Korea)', 'North Korea', inplace=True)
df.replace('The Kyrgyz Republic', 'Kyrgyzstan', inplace=True)
df.replace('Laos', "Lao People's Democratic Republic", inplace=True)
df.replace('Micronesia', 'Federated States of Micronesia', inplace=True)
df.replace('Russia', 'Russian Federation', inplace=True)
df.replace('Solomon Island', 'Solomon Islands', inplace=True)
df.replace('Syria', 'Syrian Arab Republic', inplace=True)
df.replace('The Bahamas', 'Bahamas', inplace=True)
df.replace('Cote d\'Ivoire', 'Côte d\'Ivoire', inplace=True)
df.replace('The Gambia', 'Gambia', inplace=True)
df.replace('Democratic Republic of the Congo', 'Congo, The Democratic Republic of the', inplace=True)
df.replace('Bonaire', 'Bonaire, Sint Eustatius and Saba', inplace=True)
df.replace('French West Indies', 'Guadeloupe', inplace=True)

In [231]:
def alpha3code(column):
    CODE=[]
    for country in column:
        try:
            code=pycountry.countries.lookup(country).alpha_3
           # .alpha_3 means 3-letter country code 
           # .alpha_2 means 2-letter country code
            CODE.append(code)
        except:
            CODE.append(pd.NA)
    return CODE# create a column for code 

In [232]:
df['CODE'] = alpha3code(df.Country)
df

,Country,Level,Date Updated,CODE
5,Worldwide Caution,Caution,2019-01-15,<NA>
0,Liechtenstein,Level 1: Exercise Normal Precautions,2022-04-19,LIE
111,Malawi,Level 1: Exercise Normal Precautions,2022-04-18,MWI
210,Sint Eustatius,Level 1: Exercise Normal Precautions,2022-04-18,<NA>
104,Kuwait,Level 1: Exercise Normal Precautions,2022-04-20,KWT
...,...,...,...,...
160,Ukraine,Level 4: Do Not Travel,2022-04-19,UKR
164,Venezuela,Level 4: Do Not Travel,2022-04-18,VEN
27,Hong Kong,Level 4: Do Not Travel,2022-04-25,HKG
146,Sudan,Level 4: Do Not Travel,2022-04-19,SDN


In [233]:
df.loc[df['Country'] == 'Kosovo', 'CODE'] = "KOS"

In [234]:
df.dropna(inplace=True)

In [238]:
%matplotlib notebook
fig = px.choropleth(df, locations="CODE", color="Level", hover_name="Country", color_discrete_sequence=["#003875", "#ffcc66", "#ff9900", "#ff0000", "black"])
fig.show("png")